# 01. 문자 단위 RNN(Char RNN)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [3]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
print(char_vocab)
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5


In [4]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [9]:
for i, c in enumerate(char_vocab):
    print('인덱스 : ', i)
    print('list안의 요소 : ', c)

인덱스 :  0
list안의 요소 :  !
인덱스 :  1
list안의 요소 :  a
인덱스 :  2
list안의 요소 :  e
인덱스 :  3
list안의 요소 :  l
인덱스 :  4
list안의 요소 :  p


In [10]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


 나중에 예측 결과를 다시 문자 시퀀스로 보기위해서 반대로 정수로부터 문자를 얻을 수 있는 index_to_char을 만든다.

In [15]:
index_to_char={}
for key, value in char_to_index.items():
    print('key : ',key)
    print('value : ',value)
    index_to_char[value] = key
print(index_to_char)

key :  !
value :  0
key :  a
value :  1
key :  e
value :  2
key :  l
value :  3
key :  p
value :  4
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [17]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data) # 'apple'
print(y_data) # 'pple!'

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받습니다. 그렇기 때문에 배치 차원을 추가해줍니다.

In [18]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [19]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]
